In [12]:
import numpy as np
import scipy.linalg as la
import openfermion as of
import cirq
from kcommute import compute_blocks, get_si_sets

## Bounding $||H - \tilde{H}||$

In [17]:
l = 2
t = 1.0
u = 4.0
hamiltonian_fermi = of.hamiltonians.fermi_hubbard(
    l, l, t, u, spinless=False, periodic=False
)
hamiltonian = of.transforms.jordan_wigner(hamiltonian_fermi)
hamiltonian_cirq = of.transforms.qubit_operator_to_pauli_sum(hamiltonian)

In [18]:
groups = get_si_sets(hamiltonian_cirq, k=1)
print(f"Hamiltonian has {len(groups)} groups.")

Hamiltonian has 9 groups.


In [19]:
group_psums = [sum(group) for group in groups]
print(type(group_psums[0]))

<class 'cirq.ops.linear_combinations.PauliSum'>


In [21]:
# Evaluate sum_{gamma_1=1}^Gamma ||[sum_{gamma_2=gamma_1+1}^Gamma H_gamma_2, H_gamma_1]
comm_norm = 0.
for gamma1 in range(len(group_psums)):
    h_gamma1_mat = group_psums[gamma1].matrix(hamiltonian_cirq.qubits)
    for gamma2 in range(gamma1 + 1, len(group_psums)):
        h_gamma2_mat = group_psums[gamma2].matrix(hamiltonian_cirq.qubits)
        comm = h_gamma2_mat * h_gamma1_mat - h_gamma1_mat - h_gamma2_mat
        comm_norm += la.norm(comm)
print(f"comm_norm = {comm_norm}")

h_norm = la.norm(hamiltonian_cirq.matrix())
print(f"h_norm = {h_norm}")

comm_norm = 1135.2152591239123
h_norm = 90.50966799187809


In [ ]:
d = 40 # See Ryan's F-H numerics.